In [1]:
import os
import errno
import rasterio
from pathlib import Path

# Lista los archivos con os.listdir
# La ruta son los archivos tif
folder_path_tif = '/home/farix/python/imagen/tiff/'

# Cambia la ruta para guardar las imágenes png
# Cambia tiff por png, para tener una carpeta con mismo nombres de las imágenes tif
folder_path_png = folder_path_tif.replace('tiff', 'png')

# Imprime la ruta del folder png
print(folder_path_png)

# Crea folder para guardar las imágenes png
try:
    os.mkdir(folder_path_png)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

# Recorre cada archivo en la carpeta de imágenes TIF
for entry in os.listdir(folder_path_tif):
    if os.path.isfile(os.path.join(folder_path_tif, entry)):
        print(entry)
        # Para GDAL, toma los valores máximos y mínimos de cada banda en la imagen TIF
        # Con rasterio
        RGB = os.path.join(folder_path_tif, entry)
        imagen_tif = rasterio.open(RGB)

        # Lectura del valor mínimo de la banda 1
        arrayMin1 = imagen_tif.read(1)
        banda1Min = arrayMin1.min()
        
        # Lectura del valor máximo de la banda 1
        arrayMax1 = imagen_tif.read(1)
        banda1Max = arrayMax1.max()

        # Lectura del valor mínimo de la banda 2
        arrayMin2 = imagen_tif.read(2)
        banda2Min = arrayMin2.min()

        # Lectura del valor máximo de la banda 2
        arrayMax2 = imagen_tif.read(2)
        banda2Max = arrayMax2.max()

        # Lectura del valor mínimo de la banda 3
        arrayMin3 = imagen_tif.read(3)
        banda3Min = arrayMin3.min()

        # Lectura del valor máximo de la banda 3
        arrayMax3 = imagen_tif.read(3)
        banda3Max = arrayMax3.max()

        # Conversión a PNG

        # La carpeta de salida de los archivos png
        # Archivo de salida, el nombre es el mismo que el archivo de entrada con extensión PNG
        
        without_ext = os.path.splitext(entry)[0]
        file_PNG = folder_path_png + without_ext + '.png'
        print(file_PNG)

        to_png = ('gdal_translate -ot Byte -of PNG -scale_1 ' + str(banda1Min) + ' ' + str(
            banda1Max) + ' -scale_2 ' + str(banda2Min) + ' ' + str(banda2Max) + ' -scale_3 ' + str(
            banda3Min) + ' ' + str(banda3Max) + ' -a_nodata none ' + RGB + ' ' + file_PNG)
        
        print(to_png)
        
        # Ejecuta en un bash de linux para la conversión
        os.system(to_png)

# Elimina los archivos xml generados durante la conversión
delete_xml = "rm " + folder_path_png + "*.aux.xml"
print(delete_xml)
os.system(delete_xml)


/home/farix/python/imagen/png/
block_1_1.tif
/home/farix/python/imagen/png/block_1_1.png
gdal_translate -ot Byte -of PNG -scale_1 310 1724 -scale_2 365 1451 -scale_3 484 1600 -a_nodata none /home/farix/python/imagen/tiff/block_1_1.tif /home/farix/python/imagen/png/block_1_1.png
Input file size is 1976, 2281
0...10...20...30...40...50...60...70...80...90...100 - done.
block_0_1.tif
/home/farix/python/imagen/png/block_0_1.png
gdal_translate -ot Byte -of PNG -scale_1 0 1194 -scale_2 0 950 -scale_3 0 1167 -a_nodata none /home/farix/python/imagen/tiff/block_0_1.tif /home/farix/python/imagen/png/block_0_1.png
Input file size is 1976, 2281
0...10...20...30...40...50...60...70...80...90...100 - done.
block_2_1.tif
/home/farix/python/imagen/png/block_2_1.png
gdal_translate -ot Byte -of PNG -scale_1 346 1854 -scale_2 354 1544 -scale_3 528 1547 -a_nodata none /home/farix/python/imagen/tiff/block_2_1.tif /home/farix/python/imagen/png/block_2_1.png
Input file size is 1976, 2281
0...10...20...30...4

0